In [12]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

START, END = datetime(2024,1,1), datetime(2025,1,31)

# --- 1) Try tvDatafeed with different exchange codes ---
tv = TvDatafeed()
exchanges = ["HOSE", "HSX", "VN", "UPCOM", "HNX"]
df = None

for exch in exchanges:
    try:
        tmp = tv.get_hist(
            symbol   = "MWG",
            exchange = exch,
            interval = Interval.in_daily,
            n_bars   = 400
        )
        if tmp is not None and not tmp.empty:
            df = tmp
            print(f"✅ Got data from exchange '{exch}'")
            break
    except Exception as e:
        print(f"ℹ️  tvDatafeed error for {exch}: {e}")

if df is not None and not df.empty:
    # strip timezone, filter the date window
    df.index = df.index.tz_localize(None)
    df = df.loc["2024-01-01":"2025-01-31"]
    df.to_csv("MWG_2024-01-01_to_2025-01-31_tv.csv")
    print(f"🎉 Saved {len(df)} rows to MWG_2024-01-01_to_2025-01-31_tv.csv")
else:
    print("⚠️  tvDatafeed failed for all exchanges; falling back to scraping…")

    # --- 2) Fallback: scrape Vietstock’s ‘Lịch sử giá’ table ---
    def fetch_page(page):
        url = (
          "https://finance.vietstock.vn/companies/mwg/mobile-world-group/"
          "lich-su-gia?languageid=2&page=" + str(page)
        )
        r = requests.get(url)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")
        tbl  = soup.find("table", class_="tblData")
        if not tbl:
            return []
        rows = []
        for tr in tbl.tbody.find_all("tr"):
            cols = [td.text.strip().replace(",", "") for td in tr.find_all("td")]
            dt = datetime.strptime(cols[0], "%d/%m/%Y")
            if dt < START:
                return []
            rows.append([dt, float(cols[1]), float(cols[2]), float(cols[3]), float(cols[4]), int(cols[5])])
        return rows

    all_rows = []
    page = 1
    while True:
        page_rows = fetch_page(page)
        if not page_rows:
            break
        all_rows.extend(page_rows)
        page += 1

    # Build and save DataFrame
    df2 = pd.DataFrame(all_rows, columns=["Date","Open","High","Low","Close","Volume"])
    df2 = df2.set_index("Date").sort_index()
    df2.to_csv("MWG_2024-01-01_to_2025-01-31_vietstock.csv")
    print(f"🎉 Scraped {len(df2)} rows to MWG_2024-01-01_to_2025-01-31_vietstock.csv")


✅ Got data from exchange 'HOSE'
🎉 Saved 267 rows to MWG_2024-01-01_to_2025-01-31_tv.csv


In [13]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

START, END = datetime(2024,1,1), datetime(2025,1,31)

# --- 1) Thử tvDatafeed với VN-Index ---
tv = TvDatafeed()
# Trên TradingView, VN-Index thường là symbol "VNINDEX" trên exchange "INDEX"
df = None
try:
    df = tv.get_hist(
        symbol   = "VNINDEX",
        exchange = "INDEX",
        interval = Interval.in_daily,
        n_bars   = 400
    )
    if df is not None and not df.empty:
        df.index = df.index.tz_localize(None)
        df = df.loc[START:END]
        df.to_csv("VNINDEX_2024-01-01_to_2025-01-31_tv.csv")
        print(f"🎉 Saved {len(df)} rows to VNINDEX_2024-01-01_to_2025-01-31_tv.csv")
    else:
        raise ValueError("tvDatafeed không trả về dữ liệu")
except Exception as e:
    print(f"⚠️  tvDatafeed error: {e}\n   Falling back to scraping…")

    # --- 2) Fallback: scrape Vietstock’s ‘Lịch sử giá’ của VN-Index ---
    def fetch_page(page):
        url = (
          "https://finance.vietstock.vn/ty-so/vn-index/lich-su-gia?"
          "languageid=2&page=" + str(page)
        )
        r = requests.get(url)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")
        tbl  = soup.find("table", class_="tblData")
        if not tbl:
            return []
        rows = []
        for tr in tbl.tbody.find_all("tr"):
            cols = [td.text.strip().replace(",", "") for td in tr.find_all("td")]
            dt = datetime.strptime(cols[0], "%d/%m/%Y")
            if dt < START:
                return []
            rows.append([
                dt,
                float(cols[1]),  # Open
                float(cols[2]),  # High
                float(cols[3]),  # Low
                float(cols[4]),  # Close
                int(cols[5])     # Volume
            ])
        return rows

    all_rows = []
    page = 1
    while True:
        page_rows = fetch_page(page)
        if not page_rows:
            break
        all_rows.extend(page_rows)
        page += 1

    df2 = pd.DataFrame(all_rows, columns=["Date","Open","High","Low","Close","Volume"])
    df2 = df2.set_index("Date").sort_index()
    df2.to_csv("VNINDEX_2024-01-01_to_2025-01-31_vietstock.csv")
    print(f"🎉 Scraped {len(df2)} rows to VNINDEX_2024-01-01_to_2025-01-31_vietstock.csv")


🎉 Saved 267 rows to VNINDEX_2024-01-01_to_2025-01-31_tv.csv


In [ ]:
import pandas as pd
import ace_tools as tools

# Đọc dữ liệu giá của MWG
df_mwg = pd.read_csv('MWG_2024-01-01_to_2025-01-31_tv.csv', index_col=0, parse_dates=True)

# Lọc dữ liệu cho Quý 1/2025 (ở đây dữ liệu chỉ tới 31/01/2025)
df_q1 = df_mwg.loc['2025-01-01':'2025-01-31'].copy()

# Số cổ phiếu lưu hành (outstanding shares) tại Q1/2025
shares_outstanding = 14_622_441_770_000  # cổ phiếu

# Tính Market Cap (VND) cho mỗi ngày
df_q1['market_cap'] = df_q1['close'] * shares_outstanding

# Hiển thị kết quả
tools.display_dataframe_to_user(
    name="Market Cap của MWG trong Quý 1/2025",
    dataframe=df_q1[['close', 'market_cap']].head(10)
)
